In [1]:
import os 

In [2]:
%pwd

'/home/abdelilah/Documents/projet/textsummarizer/TextSummarizer/research'

In [3]:
os.chdir("../")

In [4]:
%pwd
%ls




app.py      Dockerfile  main.py      requirements.txt  src/
artifacts/  LICENSE     params.yaml  research/         template.py
config/     logs/       README.md    setup.py


In [5]:
from dataclasses import dataclass
from pathlib import Path

# return type
@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir : Path
    source_url : str
    local_data_file : Path
    unzip_dir : Path
    

In [6]:
from textSummarizer.constants import *
from textSummarizer.utils.common import read_yaml, create_directories

In [10]:
from textSummarizer.logging import logger
class ConfigurationManager:
    def  __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        
        try:
            
            self.config = read_yaml(Path("config/config.yaml"))
        except FileNotFoundError as e:
            logger.error(f"Configuration file not found: {config_filepath}")
            raise e
        self.params = read_yaml(params_filepath)
        
        create_directories([self.config.artifacts_root])
        
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion
        
        create_directories([config.root_dir])
        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_url=config.source_url,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir,
        )
        return data_ingestion_config
    
    

In [8]:
import os 
import urllib.request as request
import zipfile
from textSummarizer.logging import logger
from textSummarizer.utils.common import get_size


In [20]:
class DataIngestion:
    def __init__(self,config: DataIngestionConfig):
        self.config = config
        
        
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_url,
                filename = self.config.local_data_file,
            )
            logger.info(f"Downloaded {filename} - with following info: {headers}")
        else:
            logger.info(f"File {get_size(Path(self.config.local_data_file))} already exists.")
        
    def extract_zip_file(self):
        """
        zip_file_path:str
        Extracts the zip file into the data directory
        Functions retuens None
        
        """  
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file,'r') as zip_ref:
            zip_ref.extractall(unzip_path)
            logger.info(f"Extracted {self.config.local_data_file} to {unzip_path}")

In [21]:
# creating piplines
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e
    

[2024-10-05 16:27:55,864: INFO:common:yaml file: config/config.yaml loaded successfully]
[2024-10-05 16:27:55,869: INFO:common:yaml file: params.yaml loaded successfully]
[2024-10-05 16:27:55,871: INFO:common:Directory created: artifacts]
[2024-10-05 16:27:55,874: INFO:common:Directory created: artifacts/data_ingestion]


[2024-10-05 16:28:03,265: INFO:2773722855:Downloaded artifacts/data_ingestion/data.zip - with following info: Connection: close
Content-Length: 24804082
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "bb7ce0fa3f7ed74c55ead276bcb35bcb4676a252ada216e46110e2dafd2d82ce"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 2787:3E9A7E:35ED712:38722B0:67015AFB
Accept-Ranges: bytes
Date: Sat, 05 Oct 2024 15:28:00 GMT
Via: 1.1 varnish
X-Served-By: cache-mad22051-MAD
X-Cache: MISS
X-Cache-Hits: 0
X-Timer: S1728142077.572350,VS0,VE3820
Vary: Authorization,Accept-Encoding,Origin
Access-Control-Allow-Origin: *
Cross-Origin-Resource-Policy: cross-origin
X-Fastly-Request-ID: 6223cc5aa71d6f76510f3ee30fdd673c9c7de43a
Expires: Sat, 05 Oct 2024 15:33:00 GMT
Source-Age: 4

]
[2024-10-05 16:28:03,885: INFO:27737228